In [16]:
import snowflake.connector
import pandas as pd
import seaborn as sns
import plotly.express as px
import warnings
import numpy as np
import matplotlib.pyplot as plt
import os

usr = os.environ['svc_user']
pwd = os.environ['svc_pwd']
# usr = 'SVC_PROMO'
# pwd = 'nBrVtHzLh8dk2Kvv'

warnings.filterwarnings('ignore')

In [17]:
def sf_connect():
    con = snowflake.connector.connect(  
        account = 'chewy.us-east-1',
        user = usr,
        password=pwd,
        autocommit = False,
        database= 'EDLDB',
        warehouse = 'IT_WH')
    return con
    
def exec_qry_all(qry):
    con = sf_connect()
    cur = con.cursor()
    try:
        cur.execute(qry,timeout=3600)
        return cur.fetch_pandas_all()   
    except Exception as e:
        print(e)
    finally:
        cur.close()

In [35]:
qry = '''WITH CUSTOMERS AS (
  SELECT DISTINCT
  CUSTOMER_ID,
  CASE WHEN ORDERS_L3M > 0 THEN 1 ELSE 0 END AS ACTIVE_3M, 
  ORDERS_L3M,
  ORDERS_L6M,
  ORDERS_L9M,
  ORDERS_L12M,
  FIRST_ORDER_DATE,
  LAST_ORDER_DATE,
  AUTOSHIP_STATUS,
  REGISTRATION_DATE
  FROM CDM.CUSTOMER_AGGREGATE
  
  WHERE REGISTRATION_DATE BETWEEN DATEADD('day',-365,CURRENT_DATE) AND CURRENT_DATE
  
),

ORDERS AS (
  
  SELECT
  CA.CUSTOMER_ID,
  CA.REGISTRATION_DATE,
  CA.FIRST_ORDER_DATE,
  OL.ORDER_ID,
  OL.PRODUCT_ID,
  PD.CATEGORY_LEVEL1,
  PD.CATEGORY_LEVEL2,
  PD.CATEGORY_LEVEL3,
  PD.MERCH_CLASSIFICATION1,
  PD.MERCH_CLASSIFICATION2,
  CAST(OL.ORDER_PLACED_DTTM AS DATE) AS ORDER_PLACED_DATE,
  OL.ORDER_LINE_TOTAL_PRICE AS ORDER_PRICE,
  CASE WHEN OL.ORDER_AUTO_REORDER_FLAG = 'TRUE' then 'Autoship' else 'Non_Autoship' end as ORDER_TYPE,
  ROW_NUMBER() OVER (PARTITION BY CA.CUSTOMER_ID, 
                     CASE WHEN OL.ORDER_AUTO_REORDER_FLAG = 'TRUE' then 'Autoship' else 'Non_Autoship' END ORDER BY OL.ORDER_PLACED_DTTM ASC) AS AS_RANKING,
  DENSE_RANK() OVER (PARTITION BY CA.CUSTOMER_ID ORDER BY OL.ORDER_PLACED_DTTM ASC) AS ORDER_RANKING
  
  FROM CUSTOMERS CA
  
  LEFT JOIN ECOM.ORDER_LINE OL
  ON CA.CUSTOMER_ID = OL.CUSTOMER_ID 
  AND OL.ORDER_LINE_TOTAL_PRICE != 0
  AND OL.ORDER_LINE_SHIPPED_DTTM IS NOT NULL
  AND OL.ORDER_STATUS NOT IN ('X', 'P', 'J')
  AND OL.ORDER_PLACED_DTTM BETWEEN DATEADD('day',-365,CURRENT_DATE) AND CURRENT_DATE
  
  LEFT JOIN PDM.PRODUCT PD
  ON OL.PRODUCT_ID = PD.PRODUCT_ID

),

MC1_DATES AS (
  SELECT
  CUSTOMER_ID,
  
  MIN(CASE WHEN MERCH_CLASSIFICATION1 = 'Consumables' THEN ORDER_PLACED_DATE ELSE NULL END) AS FIRST_CONSUMABLES_DATE,
  MIN(CASE WHEN MERCH_CLASSIFICATION1 = 'Healthcare' THEN ORDER_PLACED_DATE ELSE NULL END) AS FIRST_HEALTHCARE_DATE,
  MIN(CASE WHEN MERCH_CLASSIFICATION1 = 'Hard Goods' THEN ORDER_PLACED_DATE ELSE NULL END) AS FIRST_HARDGOODS_DATE,
  MIN(CASE WHEN MERCH_CLASSIFICATION1 = 'Specialty' THEN ORDER_PLACED_DATE ELSE NULL END) AS FIRST_SPECIALITY_DATE,
  MIN(CASE WHEN MERCH_CLASSIFICATION1 = 'Virtual Bundle' THEN ORDER_PLACED_DATE ELSE NULL END) AS FIRST_VB_DATE
  
  FROM ORDERS
  
  GROUP BY 1
),

GA_HITS AS (
  SELECT
    O.CUSTOMER_ID,

    SUM(CASE WHEN lower(EVENT_ACTION) = 'detail' AND lower(EVENT_CATEGORY) = 'eec' AND GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PDP_HITS_BETWEEN_SESSIONS,
    //SUM(CASE WHEN lower(EVENT_ACTION) = 'impression' AND lower(EVENT_LABEL) in ('search-results','browse','brand-page','deals') AND GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PLP_HITS_BETWEEN_SESSIONS,
    SUM(CASE WHEN lower(EVENT_ACTION) = 'impression' AND (PAGE_PATH_LEVEL1 like '%/b/%' or PAGE_PATH_LEVEL1 like '%/f/%' or PAGE_PATH_LEVEL1 like '%/brands/%' or PAGE_PATH_LEVEL1 like '%/deals/%' or PAGE_PATH_LEVEL1 like '%/s%') AND GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PLP_HITS_BETWEEN_SESSIONS,
    SUM(CASE WHEN lower(EVENT_ACTION) like '%add%to%cart%' AND lower(EVENT_CATEGORY) = 'eec' AND GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS ATC_HITS_BETWEEN_SESSIONS,
    MAX(CASE WHEN GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN CHANNEL_GROUPING ELSE NULL END) AS SOURCE_BETWEEN_SESSIONS,
    MAX(CASE WHEN GA_SESSIONS_DATE > FIRST_ORDER_DATE AND GA_SESSIONS_DATE < M.FIRST_CONSUMABLES_DATE THEN PRODUCT_MERCH_CLASSIFICATION2 ELSE NULL END) AS MC2_BETWEEN_SESSIONS,

    SUM(CASE WHEN lower(EVENT_ACTION) = 'detail' AND lower(EVENT_CATEGORY) = 'eec' AND GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PDP_HITS_ORDER_SESSION,
    //SUM(CASE WHEN lower(EVENT_ACTION) = 'impression' AND lower(EVENT_LABEL) in ('search-results','browse','brand-page','deals') AND GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PLP_HITS_ORDER_SESSION,
    SUM(CASE WHEN lower(EVENT_ACTION) = 'impression' AND (PAGE_PATH_LEVEL1 like '%/b/%' or PAGE_PATH_LEVEL1 like '%/f/%' or PAGE_PATH_LEVEL1 like '%/brands/%' or PAGE_PATH_LEVEL1 like '%/deals/%' or PAGE_PATH_LEVEL1 like '%/s%') AND GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS PLP_HITS_ORDER_SESSION,
    SUM(CASE WHEN lower(EVENT_ACTION) like '%add%to%cart%' AND lower(EVENT_CATEGORY) = 'eec' AND GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN 1 ELSE 0 END) AS ATC_HITS_ORDER_SESSION,
    MAX(CASE WHEN GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN CHANNEL_GROUPING ELSE NULL END) AS SOURCE_ORDER_SESSION,
    MAX(CASE WHEN GA_SESSIONS_DATE = M.FIRST_CONSUMABLES_DATE THEN PRODUCT_MERCH_CLASSIFICATION2 ELSE NULL END) AS MC2_ORDER_SESSION
  
  FROM
    ORDERS AS O
    JOIN MC1_DATES AS M ON O.CUSTOMER_ID = M.CUSTOMER_ID
    JOIN "EDLDB"."GA"."GA_SESSIONS_HITS_PRODUCTS_UNION" AS G ON O.CUSTOMER_ID = G.CUSTOMER_ID

  WHERE GA_SESSIONS_DATE BETWEEN DATEADD('day',-365,CURRENT_DATE) AND CURRENT_DATE
      AND G.CUSTOMER_ID IS NOT NULL
      AND FIRST_CONSUMABLES_DATE IS NOT NULL
      //AND O.MERCH_CLASSIFICATION1 = 'Consumables'
      AND PRODUCT_MERCH_CLASSIFICATION1 = 'Consumables'

  GROUP BY 1
)

SELECT
    C.CUSTOMER_ID,
    C.FIRST_ORDER_DATE,
    M.FIRST_CONSUMABLES_DATE,
    PDP_HITS_BETWEEN_SESSIONS,
    PLP_HITS_BETWEEN_SESSIONS,
    ATC_HITS_BETWEEN_SESSIONS,
    SOURCE_BETWEEN_SESSIONS,
    MC2_BETWEEN_SESSIONS,
    PDP_HITS_ORDER_SESSION,
    PLP_HITS_ORDER_SESSION,
    ATC_HITS_ORDER_SESSION,
    SOURCE_ORDER_SESSION,
    MC2_ORDER_SESSION

FROM
    "EDLDB"."ECOM_SANDBOX"."FO_HG_CUSTOMERS" as C
    JOIN GA_HITS AS GA ON C.CUSTOMER_ID = GA.CUSTOMER_ID
    JOIN MC1_DATES AS M ON C.CUSTOMER_ID = M.CUSTOMER_ID

WHERE
    FIRST_CONSUMABLES_DATE IS NOT NULL
'''

In [36]:
dataset = exec_qry_all(qry)
dataset.head()

,CUSTOMER_ID,FIRST_ORDER_DATE,FIRST_CONSUMABLES_DATE,PDP_HITS_BETWEEN_SESSIONS,PLP_HITS_BETWEEN_SESSIONS,ATC_HITS_BETWEEN_SESSIONS,SOURCE_BETWEEN_SESSIONS,MC2_BETWEEN_SESSIONS,PDP_HITS_ORDER_SESSION,PLP_HITS_ORDER_SESSION,ATC_HITS_ORDER_SESSION,SOURCE_ORDER_SESSION,MC2_ORDER_SESSION
0,153558315,2021-07-05,2021-10-20,0,780,0,Email,Perishable,15,330,30,Chewy Paid Search,Core Treats
1,183824213,2021-11-14,2022-01-10,48,248,20,Paid Search,Premium,0,0,0,None,None
2,161060966,2021-09-04,2021-09-16,12,324,12,Chewy Paid Search,Core Treats,0,0,0,Chewy Paid Search,Core Treats
3,151367318,2021-06-24,2021-08-29,36,5112,36,Paid Search,Premium,72,2556,54,Paid Search,Premium
4,188846192,2021-11-11,2022-01-18,14,322,7,Paid Search,Litter,0,0,0,None,None


---

First order HG customers to first order consumables - where do they come from? <br>
What does their activity look like in between sessions? <br>
What does their activity look like on the day of the first consumables order?

In [37]:
source = dataset.groupby(['CUSTOMER_ID'])['SOURCE_ORDER_SESSION'].max()
source.value_counts(normalize = True).mul(100).round(2)

Direct               26.46
Email                21.42
PLA                  13.73
Organic Search       11.34
Chewy Paid Search     9.43
Paid Search           8.54
Affiliates            5.64
Referral              1.61
Paid Social           0.79
Display               0.62
Organic Social        0.21
(Other)               0.20
Video                 0.01
Name: SOURCE_ORDER_SESSION, dtype: float64

In [38]:
d1 = dataset.groupby(['CUSTOMER_ID'])[['PDP_HITS_BETWEEN_SESSIONS','PLP_HITS_BETWEEN_SESSIONS','ATC_HITS_BETWEEN_SESSIONS','PDP_HITS_ORDER_SESSION','PLP_HITS_ORDER_SESSION','ATC_HITS_ORDER_SESSION']].first()
d1 = d1.reset_index()

d1.head()

,CUSTOMER_ID,PDP_HITS_BETWEEN_SESSIONS,PLP_HITS_BETWEEN_SESSIONS,ATC_HITS_BETWEEN_SESSIONS,PDP_HITS_ORDER_SESSION,PLP_HITS_ORDER_SESSION,ATC_HITS_ORDER_SESSION
0,4599255,80,1200,0,0,0,0
1,5089305,0,30,0,150,420,60
2,6167307,0,0,0,0,495,0
3,7000077,0,0,0,6,63,6
4,8435008,24,40,0,12,28,12


Avg hits for activity

In [39]:
cols = ['PDP_HITS_BETWEEN_SESSIONS','PLP_HITS_BETWEEN_SESSIONS','ATC_HITS_BETWEEN_SESSIONS','PDP_HITS_ORDER_SESSION','PLP_HITS_ORDER_SESSION','ATC_HITS_ORDER_SESSION']
[np.round(d1[i].mean(),2) for i in cols]

[42.35, 852.26, 11.3, 33.78, 577.14, 15.53]

---

% of users who have activity after first order and before the day of consumables purchase

In [40]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_BETWEEN_SESSIONS'] != 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_BETWEEN_SESSIONS'] != 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_BETWEEN_SESSIONS'] != 0)])/len(d1) * 100,2))

PDP 40.61
PLP 48.19
ATC 28.62


Of which - <br>
% of users who ONLY have activity after first order and before the day of consumables purchase

In [41]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_BETWEEN_SESSIONS'] != 0) & (d1['PDP_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_BETWEEN_SESSIONS'] != 0) & (d1['PLP_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_BETWEEN_SESSIONS'] != 0) & (d1['ATC_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))

PDP 23.81
PLP 25.69
ATC 21.95


---

% of users who have activity on the day of consumables purchase

In [42]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))

PDP 54.33
PLP 51.94
ATC 51.77


Of which - <br>
% of users who ONLY have activity on the day of consumables purchase

In [43]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_BETWEEN_SESSIONS'] == 0) & (d1['PDP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_BETWEEN_SESSIONS'] == 0) & (d1['PLP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_BETWEEN_SESSIONS'] == 0) & (d1['ATC_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))

PDP 37.53
PLP 29.44
ATC 45.1


---

% of users who have activity both before and on the day of consumables purchase

In [44]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_BETWEEN_SESSIONS'] != 0) & (d1['PDP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_BETWEEN_SESSIONS'] != 0) & (d1['PLP_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_BETWEEN_SESSIONS'] != 0) & (d1['ATC_HITS_ORDER_SESSION'] != 0)])/len(d1) * 100,2))

PDP 16.81
PLP 22.5
ATC 6.67


---

In [45]:
print("PDP",np.round(len(d1[(d1['PDP_HITS_BETWEEN_SESSIONS'] == 0) & (d1['PDP_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))
print("PLP",np.round(len(d1[(d1['PLP_HITS_BETWEEN_SESSIONS'] == 0) & (d1['PLP_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))
print("ATC",np.round(len(d1[(d1['ATC_HITS_BETWEEN_SESSIONS'] == 0) & (d1['ATC_HITS_ORDER_SESSION'] == 0)])/len(d1) * 100,2))

PDP 21.86
PLP 22.37
ATC 26.28
